In [ ]:
# Post-Processing
import pickle
import pandas as pd 
import numpy as np
import os

from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
X_train, X_test, y_train, y_test = pickle.load(open('mnist.pkl', 'rb'))

In [ ]:
from matplotlib import colors as mcolors
from matplotlib.colors import LinearSegmentedColormap

cmap = LinearSegmentedColormap.from_list(
    'Mei2019', 
    np.array([
        (243, 232, 29),
        (245, 173, 47),
        (140, 193, 53),
        (50,  191, 133),
        (23,  167, 198),
        (36,  123, 235),
        (53,  69,  252),
        (52,  27,  203)
    ])/255., 
    N=256
)

# cmap = cc.m_bmy

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))
fig = plt.figure(figsize=(6,.5))
img = plt.imshow(gradient, aspect='auto', cmap=cmap)
title = plt.title('Colormap stolen from Mei2019')

norm=mcolors.LogNorm()

In [ ]:
results = pickle.load(open('results_different_lambdas.pkl', 'rb')) + pickle.load(open('results_end_to_end.pkl', 'rb'))
result_df = pd.DataFrame.from_dict(results)

In [ ]:
force = lambda y,f: 1 - y*f
loss = lambda y,f: np.mean(np.maximum(0, force(y,f))**2, -1)
N_del = lambda y,f: np.sum(force(y,f) >= 0, -1)

result_df['test_loss'] = result_df.y_test_hat.apply(lambda f: loss(y_test, f))
result_df['train_loss'] = result_df.y_train_hat.apply(lambda f: loss(y_train, f))
result_df['N_del'] = result_df.y_train_hat.apply(lambda f: N_del(y_train, f))
result_df['forces'] = result_df.y_train_hat.apply(lambda f: force(y_train, f))

result_df['N/P'] = result_df['N']/result_df['P']
result_df['P/N'] = result_df['P']/result_df['N']
result_df['N_del/P'] = result_df['N_del']/result_df['P']
result_df['N_del/N'] = result_df['N_del']/result_df['N']

In [ ]:
result_end_to_end_df = result_df[result_df['lambda'] == 0]
result_df = result_df[result_df['lambda'] > 0]

In [ ]:
data = result_df.sort_values('lambda', ascending=True)
plt.scatter(data['N_del/N'], data['train_loss'], c=data['lambda'], alpha=.5, cmap=cmap.reversed(), norm=norm)
# plt.yscale('log')
plt.xscale('log')
plt.colorbar(label=r'$\lambda$ (regularization)')
plt.xlabel(r'$N_\Delta/N$')
plt.ylabel(r'Train $\mathcal{L}$')

data = result_end_to_end_df
plt.scatter(data['N_del/N'], data['train_loss'], c='k', marker='x')

In [ ]:
data = result_df.sort_values('lambda', ascending=False)
plt.scatter(data['N_del/N'], data['train_loss'], c=data['lambda'], alpha=.5, cmap=cmap.reversed(), norm=norm)
plt.yscale('log')
plt.xscale('log')
plt.colorbar(label=r'$\lambda$ (regularization)')
plt.xlabel(r'$N_\Delta/N$')
plt.ylabel(r'Train $\mathcal{L}$')

data = result_end_to_end_df.query('N < 200')
plt.scatter(data['N_del/N'], data['train_loss'], c='k', marker='x')

In [ ]:
data = result_end_to_end_df.query('N < 300')
plt.scatter(data['N_del/N'], data['train_loss'], c=data['N'], marker='x')
plt.colorbar(norm=norm)
plt.yscale('log')
plt.xscale('log')

In [ ]:
data = result_df.sort_values('lambda', ascending=True)
plt.scatter(data['N_del/N'], data['test_loss'], c=data['lambda'], alpha=.5, cmap=cmap.reversed(), norm=norm)
plt.yscale('log')
plt.xscale('log')
plt.colorbar(label=r'$\lambda$ (regularization)')
plt.xlabel(r'$N_\Delta/N$')
plt.ylabel(r'Test $\mathcal{L}$')

data = result_end_to_end_df.query('N < 200')
plt.scatter(data['N_del/N'], data['test_loss'], c='k', marker='x')

In [ ]:
data = result_df.sort_values('lambda', ascending=True)
plt.scatter(data['P/N'], data['N_del/N'], c=data['lambda'], alpha=.5, cmap=cmap.reversed(), norm=norm)
plt.colorbar(label=r'$\lambda$ (regularization)')
plt.yscale('log')
plt.xscale('log')
plt.xlabel(r'$P/N$')
plt.ylabel(r'$N_\Delta/N$')


data = result_end_to_end_df
plt.scatter(data['P/N'], data['N_del/N'], c='k', marker='x')

In [ ]:
plt.figure(figsize=(9,6))
data = result_df.sort_values('lambda', ascending=False)
data = data[data['lambda'] == 1e-5]

#invisible plot to set the limits correctly because matplotlib gets confused with log scale scatters
x = data['N_del/N']
y = data['train_loss']

plt.plot(x, y, color='none')
plt.scatter(x, y, c=data['lambda'], cmap=cmap, norm=norm, alpha=.7)
plt.colorbar(label=r'$\lambda$ (regularization)')

plt.xlabel(r'$N_\Delta/N$')
plt.ylabel(r'Train $\mathcal{L}$')

# plt.yscale('log')
plt.xscale('log')
plt.axvline(1, color='k',ls=':')

In [ ]:
plt.figure(figsize=(9,6))
data = result_df.sort_values('lambda', ascending=False)
data = data[(data['lambda'] == 1e-5) | (data['lambda'] == 1e-13 )]

#invisible plot to set the limits correctly because matplotlib gets confused with log scale scatters
x = data['N_del/N']
y = data['test_loss']

plt.plot(x, y, color='none')
plt.scatter(x, y, c=data['lambda'], cmap=cmap, norm=norm, alpha=.7)
plt.colorbar(label=r'$\lambda$ (regularization)')

plt.xlabel(r'$N_\Delta/N$')
plt.ylabel(r'Test $\mathcal{L}$')

plt.yscale('log')
plt.xscale('log')
plt.axvline(1, color='k',ls=':')
plt.axvline(21, color='k',ls=':')

In [ ]:
plt.figure(figsize=(9,6))
data = result_df.sort_values('lambda', ascending=False)
data = data[(data['lambda'] == 1e-5) | (data['lambda'] == 1e-13)]

#invisible plot to set the limits correctly because matplotlib gets confused with log scale scatters
x = data['P/N']
y = data['N_del/N']

plt.plot(x, y, color='none')
plt.scatter(x, y, c=data['lambda'], cmap=cmap, norm=norm, alpha=.7)
plt.colorbar(label=r'$\lambda$ (regularization)')

data = result_end_to_end_df.query('N < 200')
plt.scatter(data['P/N'], data['N_del/N'], c='k', marker='x')


plt.xlabel('P/N')
plt.ylabel(r'$N_\Delta/N$')

plt.yscale('log')
plt.xscale('log')
plt.axhline(1, color='k',ls=':')
plt.axhline(21, color='k',ls=':')

In [ ]:
result_df['lambda']